## Install dependencies

In [ ]:
!pip install langchain-community langchain-core pinecone fastembed google-cloud-translate langchain-google-genai langchain-pinecone bert_score

In [ ]:
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from google.colab import files
from bert_score import score

import pandas as pd
import json
from google.colab import userdata

## Initialize google translate client

In [ ]:
from google.colab import files
from google.cloud import translate_v2 as translate

google_translate_client = translate.Client.from_service_account_json("key.json")
def translate_text(text, source_language, target_language = "en"):
    translate_response = google_translate_client.translate(
        text,
        source_language=source_language,
        target_language=target_language,
    )
    return translate_response["translatedText"]

## Implement the Retrieval Chain

In [ ]:
class RetrievalChain:
    def __init__(self, embedding_model, vector_store, top_k=15):
        self.embedding_model = embedding_model
        self.vector_store = vector_store
        self.top_k = top_k

    def get_retrieved_context_ids(self, query):
        """
        Returns ranked list of retrieved context_ids (top_k)
        """
        results = self.vector_store.similarity_search_with_score(
            query,
            k=self.top_k
        )

        return [doc.metadata["context_id"] for doc, _ in results]

    def compute_recall_flags(self, gt_context_id, retrieved_ids):
        """
        Computes Recall@k flags
        """
        return {
            "r@1": int(gt_context_id in retrieved_ids[:1]),
            "r@3": int(gt_context_id in retrieved_ids[:3]),
            "r@5": int(gt_context_id in retrieved_ids[:5]),
            "r@10": int(gt_context_id in retrieved_ids[:10]),
            "r@15": int(gt_context_id in retrieved_ids[:15]),
        }

    def run(self, question_id, query, query_language, gt_context_id):
        en_query = translate_text(query, query_language)
        retrieved_ids = self.get_retrieved_context_ids(en_query)

        recall_flags = self.compute_recall_flags(
            gt_context_id,
            retrieved_ids
        )

        return {
            "question_id": question_id,
            "ground_truth": gt_context_id,
            "translated_question": en_query,
            **recall_flags,
            "contexts": "|".join(map(str, retrieved_ids))
        }


## Initiaize the RetrievalChain

In [ ]:
embedding_model = FastEmbedEmbeddings()

pc_index_name = "gic-fastembed"
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))
index = pc.Index(pc_index_name)
vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

retrieval_chain = RetrievalChain(
    embedding_model=embedding_model,
    vector_store=vector_store,
)

In [ ]:
experiment_id = "qt_google_translate"

## Load test data

In [ ]:
import pandas as pd
qa_data = pd.read_csv("gic_qa_with_ids.csv")
qa_data.head(1)

## tamil questions

In [ ]:
results_ta = []

for _, row in qa_data.iterrows():
    # if row['question_id'] == 2:
    #   break
    print(f"Processing question: {row['question_id']}")
    # print(f"Question: {row['question_ta']}")
    result = retrieval_chain.run(
        question_id=row["question_id"],
        query=row["question_ta"],
        query_language="ta",
        gt_context_id=row["context_id"]
    )
    # print(result)
    # print("translated_question: ", row['translated_question'])
    # print("Ground truth: ", row["context_id"])
    # print("Retrieved: ", result["contexts"])
    results_ta.append(result)

In [ ]:
df = pd.DataFrame(results_ta)
df.tail(5)

In [ ]:
df.to_csv(f"result_ta_{experiment_id}.csv", index=False)

In [ ]:
summary_baseline_ta = {
    "Recall@1": df["r@1"].mean(),
    "Recall@3": df["r@3"].mean(),
    "Recall@5": df["r@5"].mean(),
    "Recall@10": df["r@10"].mean(),
    "Recall@15": df["r@15"].mean(),
}

summary_baseline_ta

In [ ]:
summary_pct_baseline_ta = {
    k: v * 100
    for k, v in summary_baseline_ta.items()
}

summary_pct_baseline_ta

In [ ]:
summary_pct_baseline_ta['language'] = "ta"
summary_pct_baseline_ta

## Sinhala questions

In [ ]:
results_si = []

for _, row in qa_data.iterrows():
    # if row['question_id'] == 2:
    #   break
    print(f"Processing question: {row['question_id']}")
    # print(f"Question: {row['question_si']}")
    result = retrieval_chain.run(
        question_id=row["question_id"],
        query=row["question_si"],
        query_language="si",
        gt_context_id=row["context_id"]
    )
    # print(result)
    # print("translated_question: ", row['translated_question'])
    # print("Ground truth: ", row["context_id"])
    # print("Retrieved: ", result["contexts"])
    results_si.append(result)

In [ ]:
df = pd.DataFrame(results_si)
print(len(df))
df.tail(5)

In [ ]:
df.to_csv(f"result_si_{experiment_id}.csv", index=False)

In [ ]:
summary_baseline_si = {
    "Recall@1": df["r@1"].mean(),
    "Recall@3": df["r@3"].mean(),
    "Recall@5": df["r@5"].mean(),
    "Recall@10": df["r@10"].mean(),
    "Recall@15": df["r@15"].mean(),
}

summary_baseline_si

In [ ]:
summary_pct_baseline_si = {
    k: v * 100
    for k, v in summary_baseline_si.items()
}

summary_pct_baseline_si

In [ ]:
summary_pct_baseline_si['language'] = "si"
summary_pct_baseline_si

## Overall summary

In [ ]:
summary_baseline = pd.DataFrame([summary_pct_baseline_ta, summary_pct_baseline_si])
summary_baseline

In [ ]:
cols = ["language"] + [c for c in summary_baseline.columns if c != "language"]
summary_baseline = summary_baseline[cols]
summary_baseline

In [ ]:
summary_baseline.to_csv(f"summary_{experiment_id}.csv", index=False)